## XML-Based Data Extraction from IRS Website

-This Notebook is a XML-based scraper that is used extract financial data from IRS's Form 990 Series (e-file) XML files for the year 2023. For the Opportunity Hackathon, and to provide insight on average salary.

In [1]:
# Importing needed libraries for the API Calls projects
import os # Since we downloaded the data from IRS's website we use os to read local data
import xml.etree.ElementTree as ET # Using xml.etree to parse the xml data
import pandas as pd # Good ol' pandas

The XML-based scraper is designed to extract financial data from IRS's Form 990 Series (e-file) XML files. This data extraction process allows for further analysis and reporting related to nonprofit organizations.

### Usage

- In order to use this notebook Ensure that you have downloaded the XML data from the IRS website and unzipped it. The data should be stored in the specified directory that you should set the path to.

- The Link: [IRS Form 990 Series (e-file) XML format for the year 2023](https://www.irs.gov/charities-non-profits/form-990-series-downloads)

In [2]:
# Ok since we arent including the xml data in the github repo, here is where we got it from:
# https://www.irs.gov/charities-non-profits/form-990-series-downloads
# We got this from Form 990 Series (e-file) XML format, for the year 2023, than unzipped and stored it.
DataDirectory = "/home/dark/GitHub/Opportunity-Hack-KSJ/Data"


The code performs the following key tasks during the data extraction process:

- Extracts EIN, Gross Revenue, Total Expenses, Net Revenue, Tax Period Year, and Form Type from the XML data.
- Handles different Form Types ("990" and "990EZ") by parsing the XML data accordingly.
- Collects names and titles of individuals associated with the organization based on Form Type.
- Performs additional calculations, including Total Average Salaries.


In [3]:
listData = [] # Storing data here than converting to df later
# num = 50

# So this code snippet
for folderName in os.listdir(DataDirectory): # We load the data with all the xml files
    folderPath = os.path.join(DataDirectory, folderName) # Set the path
    if os.path.isdir(folderPath): # If os path is dir of folderPath than continue
        for xmlName in os.listdir(folderPath):
            if xmlName.endswith(".xml"):
                namespace = {'ns': 'http://www.irs.gov/efile'}
                xmlPath = os.path.join(folderPath, xmlName)
                # print(xmlPath)
                tree = ET.parse(xmlPath)
                root = tree.getroot()

                # Find the EIN value in the XML
                einElem = root.find(".//ns:EIN", namespaces=namespace)
                if einElem is not None:
                    einVal = einElem.text
                else:
                    einVal = None

                # Find the Gross_Revenue value in the XML
                grossRevenueElem = root.find(".//ns:CYRevenuesLessExpensesAmt", namespaces=namespace)
                if grossRevenueElem is not None:
                    grossRevenueVal = grossRevenueElem.text
                else:
                    # If CYRevenuesLessExpensesAmt is null, try TotalRevenueAmt as an alternative
                    totalRevenueElem = root.find(".//ns:TotalRevenueAmt", namespaces=namespace)
                    if totalRevenueElem is not None:
                        grossRevenueVal = totalRevenueElem.text
                    else:
                        grossRevenueVal = 0

                # Find the Total_Expenses value in the XML
                totalExpensesElem = root.find(".//ns:CYTotalExpensesAmt", namespaces=namespace)
                if totalExpensesElem is not None:
                    totalExpensesVal = totalExpensesElem.text
                else:
                    # If CYTotalExpensesAmt is null, try TotalExpensesAmt as an alternative
                    totalExpensesElem = root.find(".//ns:TotalExpensesAmt", namespaces=namespace)
                    if totalExpensesElem is not None:
                        totalExpensesVal = totalExpensesElem.text
                    else:
                        # If TotalExpensesAmt is also null, try ExpensesAndContributionsAmt
                        expensesAndContributionsElem = root.find(".//ns:ExpensesAndContributionsAmt", namespaces=namespace)
                        if expensesAndContributionsElem is not None:
                            totalExpensesVal = expensesAndContributionsElem.text
                        else:
                            totalExpensesVal = 0

                # Calculate the Net_Revenue
                if grossRevenueVal is not None:
                    grossRevenueVal = int(grossRevenueVal)
                else:
                    grossRevenueVal = 0

                if totalExpensesVal is not None:
                    totalExpensesVal = int(totalExpensesVal)
                else:
                    totalExpensesVal = 0

                netRevenueVal = grossRevenueVal - totalExpensesVal

                # Find the Tax_Period_Year from ReturnTs or hardcode to 2023
                returnTsElem = root.find(".//ns:ReturnTs", namespaces=namespace)
                if returnTsElem is not None:
                    taxPeriodYear = returnTsElem.text[:4]
                else:
                    taxPeriodYear = "2023"

                # Find Form Type and exclude 990PF records
                returnTypeElem = root.find(".//ns:ReturnTypeCd", namespaces=namespace)
                if returnTypeElem is not None:
                    formType = returnTypeElem.text
                    if formType.lower() == "990pf":
                        continue  # Exclude 990PF records
                else:
                    formType = None

                # Initialize lists to store names and titles
                names = []
                titles = []

                # Check the form type and extract data accordingly
                if formType == "990EZ":
                    # Handle 990EZ structure with OfficerDirectorTrusteeEmplGrp
                    individuals = root.findall(".//ns:OfficerDirectorTrusteeEmplGrp", namespaces=namespace)
                    for ind in individuals:
                        personName = ind.find(".//ns:PersonNm", namespaces=namespace)
                        
                        # Handle cases where PersonNm is null by looking at the structure you provided
                        if personName is None:
                            personName = ind.find(".//ns:TitleTxt/../../ns:PersonNm", namespaces=namespace)
                        
                        if personName is not None:
                            names.append(personName.text)
                        title = ind.find(".//ns:TitleTxt", namespaces=namespace)
                        if title is not None:
                            titles.append(title.text)
                else:
                    # Handle 990 structure with Form990PartVIISectionAGrp
                    individuals = root.findall(".//ns:Form990PartVIISectionAGrp", namespaces=namespace)
                    for ind in individuals:
                        personName = ind.find(".//ns:PersonNm", namespaces=namespace)
                        title = ind.find(".//ns:TitleTxt", namespaces=namespace)
                        
                        if personName is not None:
                            names.append(personName.text)
                        if title is not None:
                            titles.append(title.text)

                name1 = names[0] if len(names) > 0 else None
                title1 = titles[0] if len(titles) > 0 else None
                name2 = names[1] if len(names) > 1 else None
                title2 = titles[1] if len(titles) > 1 else None
                name3 = names[2] if len(names) > 2 else None
                title3 = titles[2] if len(titles) > 2 else None

                try:
                    salariesDirectorsElem = root.find(".//ns:CompCurrentOfcrDirectorsGrp", namespaces=namespace)
                    salariesDirectorsVal = salariesDirectorsElem.find(".//ns:TotalAmt", namespaces=namespace).text if salariesDirectorsElem is not None else None
                except AttributeError:
                    salariesDirectorsVal = None

                try:
                    otherSalariesElem = root.find(".//ns:OtherSalariesAndWagesGrp", namespaces=namespace)
                    otherSalariesVal = otherSalariesElem.find(".//ns:TotalAmt", namespaces=namespace).text if otherSalariesElem is not None else None
                except AttributeError:
                    otherSalariesVal = None

                try:
                    otherEmployeeBenefitsElem = root.find(".//ns:OtherEmployeeBenefitsGrp", namespaces=namespace)
                    otherEmployeeBenefitsVal = otherEmployeeBenefitsElem.find(".//ns:TotalAmt", namespaces=namespace).text if otherEmployeeBenefitsElem is not None else None
                except AttributeError:
                    otherEmployeeBenefitsVal = None

                try:
                    totalPeopleElem = root.find(".//ns:TotalEmployeeCnt", namespaces=namespace)
                    totalPeopleVal = totalPeopleElem.text if totalPeopleElem is not None else None
                except AttributeError:
                    totalPeopleVal = None

                # Append all extracted data to the list
                listData.append({
                    "EIN": einVal,
                    "Gross_Revenue": grossRevenueVal,
                    "Total_Expenses": totalExpensesVal,
                    "Net_Revenue": netRevenueVal,
                    "Tax_Period_Year": taxPeriodYear,
                    "Form Type": formType,
                    "Name_1": name1,
                    "Title_1": title1,
                    "Name_2": name2,
                    "Title_2": title2,
                    "Name_3": name3,
                    "Title_3": title3,
                    "Salaries_Directors": salariesDirectorsVal,
                    "Other_Salaries": otherSalariesVal,
                    "Other_Employee_Benefits": otherEmployeeBenefitsVal,
                    "Total_People": totalPeopleVal
                })

df = pd.DataFrame(listData)

In [ ]:
df["Total_Average_Salaries"] = (df["Salaries_Directors"].astype(float) + df["Other_Salaries"].astype(float) + df["Other_Employee_Benefits"].astype(float)) / df["Total_People"].astype(float)

df.to_csv("../OpportunityHack/Data/XML_Data_IRS_2023.csv", index=False)